<a href="https://colab.research.google.com/github/KwameCode27/Fenteng_Michael_FinalProject/blob/main/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

  Using cached filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached huggingface_hub-0.34.4-py3-none-any.whl.metadata (14 kB)
     ---------------------------------------- 0.0/76.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/76.3 kB ? eta -:--:--
     --------------- ---------------------- 30.7/76.3 kB 660.6 kB/s eta 0:00:01
     -------------------- ----------------- 41.0/76.3 kB 495.5 kB/s eta 0:00:01
     ------------------------------ ------- 61.4/76.3 kB 656.4 kB/s eta 0:00:01
     ----------------------------------- -- 71.7/76.3 kB 491.5 kB/s eta 0:00:01
     ----------------------------------- -- 71.7/76.3 kB 491.5 kB/s eta 0:00:01
     -------------------------------------- 76.3/76.3 kB 325.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/494.8 kB ? eta -:--:--
   ----- ---------------------------------- 71.7/494.8 kB 2.0 MB/s eta 0:00:01
   -------- ------------------------------- 102.4/494.8 kB 1.5 MB/s eta 0:00:01
   -----

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from datasets import load_dataset

dataset = load_dataset("michsethowusu/hausa-sentiments-corpus")

# or load the separate splits if the dataset has train/validation/test splits
dataset['train'].to_csv('train_hausa.csv', index=False)

Creating CSV from Arrow format: 100%|██████████| 670/670 [00:07<00:00, 88.27ba/s] 


55186229

In [ ]:
!pip install transformers datasets scikit-learn torch matplotlib seaborn -q

In [ ]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# === Cell 2: Imports ===
import pandas as pd
import numpy as np
# import torch
# from datasets import load_dataset, DatasetDict, Dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from sklearn.model_selection import train_test_split


In [ ]:
# Load your dataset
df = pd.read_csv("/content/train_hausa.csv")

print(df.head())
print(df['label'].value_counts())  # check class balance


                                                text     label  \
0  Kuma lalle ne gidan Lahira shi ne mafi alheri ...  Negative   
1          Gaskantawa da Yesu ma bangare ne na tuba.  Negative   
2       Tafkin wutan nan kuwa shi ne mutuwa ta biyu.  Negative   
3                  A sunana, Za su fitar da aljannu.  Positive   
4  Yahi Dhoka Duniya Chalati Hai Yahi Sara Game Hai,  Positive   

   __index_level_0__  
0                  0  
1                  1  
2                  2  
3                  3  
4                  4  
label
Positive    353111
Negative    316826
Name: count, dtype: int64


In [ ]:

train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

print("Train size:", len(train_df))
print("Val size:", len(val_df))
print("Test size:", len(test_df))


Train size: 535949
Val size: 66994
Test size: 66994


In [ ]:
# Baseline (TF-IDF + Logistic Regression)
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_df['text'])
X_val = vectorizer.transform(val_df['text'])
X_test = vectorizer.transform(test_df['text'])

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, train_df['label'])

y_pred = clf.predict(X_test)

print("Baseline Results (TF-IDF + Logistic Regression):")
print(classification_report(test_df['label'], y_pred))
print("Confusion Matrix:\n", confusion_matrix(test_df['label'], y_pred))


Baseline Results (TF-IDF + Logistic Regression):
              precision    recall  f1-score   support

    Negative       0.48      0.24      0.32     31683
    Positive       0.53      0.77      0.63     35311

    accuracy                           0.52     66994
   macro avg       0.50      0.50      0.47     66994
weighted avg       0.51      0.52      0.48     66994

Confusion Matrix:
 [[ 7645 24038]
 [ 8277 27034]]


In [ ]:
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# Convert Pandas → HuggingFace Dataset
# Drop the '__index_level_0__' column before converting to Dataset
train_ds = Dataset.from_pandas(train_df.drop('__index_level_0__', axis=1))
val_ds = Dataset.from_pandas(val_df.drop('__index_level_0__', axis=1))
test_ds = Dataset.from_pandas(test_df.drop('__index_level_0__', axis=1))
dataset_dict = DatasetDict({"train": train_ds, "validation": val_ds, "test": test_ds})

# Tokenizer & model
model_name = "bert-base-multilingual-cased"  # or "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset_dict.map(preprocess, batched=True)

# Map labels to integers
unique_labels = df['label'].unique()
label_map = {label: i for i, label in enumerate(unique_labels)}

def map_labels_to_int(examples):
    examples['label'] = label_map[examples['label']]
    return examples

tokenized_datasets = tokenized_datasets.map(map_labels_to_int)


model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(unique_labels)
)

# Compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro")
    }

# Training setup
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train model
trainer.train()

# Evaluate on test set
results = trainer.predict(tokenized_datasets["test"])
print(results.metrics)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

Map:   0%|          | 0/66994 [00:00<?, ? examples/s]

Map:   0%|          | 0/66994 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import pandas as pd # Import pandas to ensure test_df is available

# Re-create datasets and tokenize
train_ds = Dataset.from_pandas(train_df.drop('__index_level_0__', axis=1))
val_ds = Dataset.from_pandas(val_df.drop('__index_level_0__', axis=1))
test_ds = Dataset.from_pandas(test_df.drop('__index_level_0__', axis=1))
dataset_dict = DatasetDict({"train": train_ds, "validation": val_ds, "test": test_ds})

model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset_dict.map(preprocess, batched=True)

unique_labels = df['label'].unique()
label_map = {label: i for i, label in enumerate(unique_labels)}

def map_labels_to_int(examples):
    examples['label'] = label_map[examples['label']]
    return examples

tokenized_datasets = tokenized_datasets.map(map_labels_to_int)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(unique_labels)
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro")
    }

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train model
trainer.train()

# Evaluate on test set
results = trainer.predict(tokenized_datasets["test"])


print("Baseline Results (TF-IDF + Logistic Regression):")
# Assuming y_pred and test_df['label'] are still available from the previous baseline execution
# If not, you would need to re-run the baseline code or load the predictions/labels.
# For now, assuming they are available.
print(classification_report(test_df['label'], y_pred))
print("Confusion Matrix:\n", confusion_matrix(test_df['label'], y_pred))

print("\nHugging Face Model Results (Test Set):")
print(results.metrics)

Map:   0%|          | 0/535949 [00:00<?, ? examples/s]

Map:   0%|          | 0/66994 [00:00<?, ? examples/s]

Map:   0%|          | 0/66994 [00:00<?, ? examples/s]

Map:   0%|          | 0/535949 [00:00<?, ? examples/s]

Map:   0%|          | 0/66994 [00:00<?, ? examples/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-222885451.py:57: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fentengkwame3 (fentengkwame3-university-of-ghana) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
